In [3]:
!pip3 install pandas
import pandas as pd

In [4]:
car_sales_df = pd.read_csv("data/car_sales.csv")

In [6]:
car_sales_df.dtypes

customer_id      int64
model           object
saleprice      float64
sale_date       object
VIN             object
dtype: object

In [10]:
car_sales_df["sale_date"] = pd.to_datetime(car_sales_df.sale_date, infer_datetime_format=True)

In [12]:
car_sales_df = car_sales_df.sort_values(by="sale_date")

In [20]:
car_sales_df_historical = car_sales_df[car_sales_df["sale_date"] < "07-01-2020"]

In [23]:
car_sales_df_batch_1 = car_sales_df[(car_sales_df["sale_date"] > "07-01-2020") & (car_sales_df["sale_date"] < "10-01-2020")]

In [24]:
car_sales_df_batch_2 = car_sales_df[(car_sales_df["sale_date"] > "10-01-2020")]

In [34]:
car_sales_df_historical.to_csv("data/historical_car_sales.csv", index=False)

In [35]:
car_sales_df_batch_1.to_csv("data/10012020_car_sales.csv", index=False)

In [36]:
car_sales_df_batch_2.to_csv("data/12312020_car_sales.csv", index=False)

In [37]:
car_sales_df_historical.columns

Index(['customer_id', 'model', 'saleprice', 'sale_date', 'VIN'], dtype='object')

In [45]:
username= 'test_user'

In [42]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
import sys

data_lake_name = "s3a://go01-demo/"

# Your Username Here:
username = "user"

spark = SparkSession \
    .builder \
    .appName("Car Sales Report") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")\
    .config("spark.sql.adaptive.enabled", "false")\
    .config("spark.yarn.access.hadoopFileSystems", data_lake_name)\
    .getOrCreate()

In [44]:
car_sales = spark.read.csv("data/car_sales.csv", header=True, inferSchema=True)

In [51]:
car_sales.dtypes

[('customer_id', 'int'),
 ('model', 'string'),
 ('saleprice', 'double'),
 ('sale_date', 'string'),
 ('VIN', 'string')]

In [56]:
car_sales.select("customer_id").tail(1)[0][0]

10000

In [57]:
car_sales.select("customer_id").head(1)[0][0]

1

In [47]:
#---------------------------------------------------
#       SQL CLEANUP: DATABASES, TABLES, VIEWS
#---------------------------------------------------
print("JOB STARTED...")
spark.sql("DROP DATABASE IF EXISTS {}_CAR_DATA CASCADE".format(username))
print("\tDROP DATABASE(S) COMPLETED")

##---------------------------------------------------
##                 CREATE DATABASES
##---------------------------------------------------
spark.sql("CREATE DATABASE {}_CAR_DATA".format(username))
print("\tCREATE DATABASE(S) COMPLETED")

JOB STARTED...
	DROP DATABASE(S) COMPLETED
	CREATE DATABASE(S) COMPLETED


In [48]:
car_sales.write.mode("overwrite").saveAsTable('{}_CAR_DATA.CAR_SALES'.format(username), format="parquet")

In [49]:
spark.sql("ALTER TABLE {}_CAR_DATA.CAR_SALES DROP COLUMN CUSTOMER_ID".format(username))

AnalysisException: DROP COLUMN is only supported with v2 tables.

In [ ]:
#---------------------------------------------------
#               LOAD ICEBERG TABLES AS DATAFRAMES
#---------------------------------------------------
car_sales_df = spark.sql("SELECT * FROM spark_catalog.{}_CAR_DATA.CAR_SALES".format(username))